In [17]:
import os

In [18]:
from google.colab import userdata
api = userdata.get('GOOGLE_API_KEY')

In [19]:
!pip install langchain langchain_google_genai langchain_core requests

In [20]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [21]:
# create tool

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two integers"""
    return a * b

In [22]:
print(multiply.invoke({'a':2, 'b':3}))

6


In [23]:
multiply.name

'multiply'

In [24]:
multiply.description

'Multiply two integers'

In [25]:
multiply.args

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

In [29]:
# tool binding

llm = ChatGoogleGenerativeAI(
    model = 'gemini-2.0-flash',
    api_key = api
)

In [30]:
llm_with_tools = llm.bind_tools([multiply])

In [31]:
llm_with_tools.invoke('how are you?')

AIMessage(content='I am doing well, thank you for asking. How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--53f2495e-5bff-4dcc-bd65-b3617606c236-0', usage_metadata={'input_tokens': 15, 'output_tokens': 18, 'total_tokens': 33, 'input_token_details': {'cache_read': 0}})

In [36]:
# tool calling

In [32]:
llm_with_tools.invoke('multiply 19 with 7')

AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 19.0, "b": 7.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--b8b2b74e-7249-47ef-9f29-6d1c633ffc97-0', tool_calls=[{'name': 'multiply', 'args': {'a': 19.0, 'b': 7.0}, 'id': '98edc61f-3ee5-4950-ada2-bf5ceedaee92', 'type': 'tool_call'}], usage_metadata={'input_tokens': 18, 'output_tokens': 5, 'total_tokens': 23, 'input_token_details': {'cache_read': 0}})

In [35]:
llm_with_tools.invoke('multiply 19 with 7').tool_calls[0]

{'name': 'multiply',
 'args': {'a': 19.0, 'b': 7.0},
 'id': '6c59066c-7ee1-483d-9ffd-73cc6fb6a033',
 'type': 'tool_call'}

In [37]:
# tool execution

In [38]:
result = llm_with_tools.invoke('multiply 19 with 7')

In [39]:
result.tool_calls[0]['args']

{'a': 19.0, 'b': 7.0}

In [40]:
multiply.invoke(result.tool_calls[0]['args'])

133

In [42]:
result.tool_calls[0]

{'name': 'multiply',
 'args': {'a': 19.0, 'b': 7.0},
 'id': '5edcd390-ff8d-4408-95d2-e393a39a391c',
 'type': 'tool_call'}

In [41]:
multiply.invoke(result.tool_calls[0])

ToolMessage(content='133', name='multiply', tool_call_id='5edcd390-ff8d-4408-95d2-e393a39a391c')

In [43]:
# formal flow

In [59]:
querry = HumanMessage('can you multiply 786 with 19?')

In [60]:
messages = [querry]

In [61]:
messages

[HumanMessage(content='can you multiply 786 with 19?', additional_kwargs={}, response_metadata={})]

In [62]:
result = llm_with_tools.invoke(messages)

In [63]:
result

AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 786.0, "b": 19.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--b6608400-1bb6-4109-b2cb-b3a2fc38f9c2-0', tool_calls=[{'name': 'multiply', 'args': {'a': 786.0, 'b': 19.0}, 'id': '3e5bc5dc-cfc5-45e9-b630-76e9b148d513', 'type': 'tool_call'}], usage_metadata={'input_tokens': 23, 'output_tokens': 5, 'total_tokens': 28, 'input_token_details': {'cache_read': 0}})

In [64]:
messages.append(result)

In [65]:
messages

[HumanMessage(content='can you multiply 786 with 19?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 786.0, "b": 19.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--b6608400-1bb6-4109-b2cb-b3a2fc38f9c2-0', tool_calls=[{'name': 'multiply', 'args': {'a': 786.0, 'b': 19.0}, 'id': '3e5bc5dc-cfc5-45e9-b630-76e9b148d513', 'type': 'tool_call'}], usage_metadata={'input_tokens': 23, 'output_tokens': 5, 'total_tokens': 28, 'input_token_details': {'cache_read': 0}})]

In [66]:
tool_result = multiply.invoke(result.tool_calls[0])

In [67]:
tool_result

ToolMessage(content='14934', name='multiply', tool_call_id='3e5bc5dc-cfc5-45e9-b630-76e9b148d513')

In [68]:
messages.append(tool_result)

In [69]:
messages

[HumanMessage(content='can you multiply 786 with 19?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 786.0, "b": 19.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--b6608400-1bb6-4109-b2cb-b3a2fc38f9c2-0', tool_calls=[{'name': 'multiply', 'args': {'a': 786.0, 'b': 19.0}, 'id': '3e5bc5dc-cfc5-45e9-b630-76e9b148d513', 'type': 'tool_call'}], usage_metadata={'input_tokens': 23, 'output_tokens': 5, 'total_tokens': 28, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='14934', name='multiply', tool_call_id='3e5bc5dc-cfc5-45e9-b630-76e9b148d513')]

In [70]:
llm_with_tools.invoke(messages)

AIMessage(content='The result of multiplying 786 by 19 is 14934.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--ec681a87-58b5-4905-ae67-1e3c2a8545d9-0', usage_metadata={'input_tokens': 31, 'output_tokens': 21, 'total_tokens': 52, 'input_token_details': {'cache_read': 0}})

In [71]:
llm_with_tools.invoke(messages).content

'The result of multiplying 786 by 19 is 14934.'

# Currency Conversion Tool

In [145]:
# create tools
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_currency_difference(base_currency: str, target_currency: str) -> float:
    """Get the currency conversion factor between the given base and target currency"""

    url = f'https://v6.exchangerate-api.com/v6/c754eab14ffab33112e380ca/pair/{base_currency}/{target_currency}'

    response = requests.get(url)

    return response.json()



@tool
def converter(base_currency: float, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
    """Convert the given amount from the base currency to the target currency"""
    return base_currency * conversion_rate


In [127]:
get_currency_difference.invoke({'base_currency': 'USD', 'target_currency': 'PKR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1753401601,
 'time_last_update_utc': 'Fri, 25 Jul 2025 00:00:01 +0000',
 'time_next_update_unix': 1753488001,
 'time_next_update_utc': 'Sat, 26 Jul 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'PKR',
 'conversion_rate': 284.8725}

In [128]:
converter.invoke({'base_currency':100, 'conversion_rate':284.8725})

28487.25

In [146]:
# tool binding

llm = ChatGoogleGenerativeAI(
    model = 'gemini-2.0-flash',
    api_key = api
)

llm_with_tools = llm.bind_tools([get_currency_difference, converter])

In [130]:
messages = [HumanMessage('What is the conversion rate between dirham and pakistani rupees? and based on that convert 100 dirhams to pakistani rupees?')]

In [131]:
messages

[HumanMessage(content='What is the conversion rate between dirham and pakistani rupees? and based on that convert 100 dirhams to pakistani rupees?', additional_kwargs={}, response_metadata={})]

In [147]:
ai_msg = llm_with_tools.invoke(messages)

In [148]:
ai_msg

AIMessage(content='The conversion rate between AED and PKR is 77.6001. I will now convert 100 dirhams to pakistani rupees.', additional_kwargs={'function_call': {'name': 'converter', 'arguments': '{"base_currency": 100.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--4d9c649b-2f5b-4834-afc0-aee950a0cdd7-0', tool_calls=[{'name': 'converter', 'args': {'base_currency': 100.0}, 'id': '5328d2a9-96c0-4ef7-81c3-4e48cc7be16e', 'type': 'tool_call'}], usage_metadata={'input_tokens': 228, 'output_tokens': 39, 'total_tokens': 267, 'input_token_details': {'cache_read': 0}})

In [149]:
messages.append(ai_msg)

In [150]:
messages

[HumanMessage(content='What is the conversion rate between dirham and pakistani rupees? and based on that convert 100 dirhams to pakistani rupees?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_currency_difference', 'arguments': '{"target_currency": "PKR", "base_currency": "AED"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--d40213cb-3918-4d59-b51f-bc3711b0fe45-0', tool_calls=[{'name': 'get_currency_difference', 'args': {'target_currency': 'PKR', 'base_currency': 'AED'}, 'id': 'e7a4ee00-867c-4c63-a98d-2112a730a419', 'type': 'tool_call'}], usage_metadata={'input_tokens': 82, 'output_tokens': 14, 'total_tokens': 96, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='{"result": "success", "documentation": "https://www.exchangerate-api.com/docs", "terms_of_use": "https://www.

In [151]:
ai_msg.tool_calls

[{'name': 'converter',
  'args': {'base_currency': 100.0},
  'id': '5328d2a9-96c0-4ef7-81c3-4e48cc7be16e',
  'type': 'tool_call'}]

In [152]:
import json

for tool_call in ai_msg.tool_calls:
  if tool_call['name']== 'get_currency_difference':
    tool_msg1 = get_currency_difference.invoke(tool_call)
    # print(tool_msg1)
    # fetch conversion rate
    conversion_rate = json.loads(tool_msg1.content)['conversion_rate']
    messages.append(tool_msg1)

  if tool_call['name']== 'converter':
    # fetch the current args
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_msg2 = converter.invoke(tool_call)
    messages.append(tool_msg2)


In [153]:
messages

[HumanMessage(content='What is the conversion rate between dirham and pakistani rupees? and based on that convert 100 dirhams to pakistani rupees?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_currency_difference', 'arguments': '{"target_currency": "PKR", "base_currency": "AED"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--d40213cb-3918-4d59-b51f-bc3711b0fe45-0', tool_calls=[{'name': 'get_currency_difference', 'args': {'target_currency': 'PKR', 'base_currency': 'AED'}, 'id': 'e7a4ee00-867c-4c63-a98d-2112a730a419', 'type': 'tool_call'}], usage_metadata={'input_tokens': 82, 'output_tokens': 14, 'total_tokens': 96, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='{"result": "success", "documentation": "https://www.exchangerate-api.com/docs", "terms_of_use": "https://www.

In [154]:
llm_with_tools.invoke(messages)

AIMessage(content='The conversion rate between Dirham and Pakistani Rupees is 1 AED = 77.6001 PKR. Therefore, 100 dirhams is equal to 7760.01 Pakistani Rupees.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--ec3383b9-ba00-44d5-8324-141cc2dc687a-0', usage_metadata={'input_tokens': 240, 'output_tokens': 50, 'total_tokens': 290, 'input_token_details': {'cache_read': 0}})

In [155]:
llm_with_tools.invoke(messages).content

'The conversion rate between Dirham and Pakistani Rupees is 1 AED = 77.6001 PKR. Therefore, 100 Dirhams is equal to 7760.01 Pakistani Rupees.'